In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *
from brute_force import planes_def

In [2]:
#%matplotlib widget
#matplotlib inline
%matplotlib qt
#%matplotlib gtk

import matplotlib.pyplot as plt
import matplotlib.font_manager

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'

from matplotlib.backends.backend_pgf import FigureCanvasPgf
matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
data = load_data(2)
d = convert(data)

print(d['C'][:,3])
print(d['E'][:,3])
print(d['o'][:,3])

P1, P2, Rsqu, Rsqu_P1, Rsqu_P2 = planes_def(data,d)
print(Rsqu, Rsqu_P1, Rsqu_P2)

[ 70.3  72.7 109.3 113.  115. ]
[]
[ 80.7  82.7  81.7  85.3  92.3 100.7  99.3 102.  107.3 106.3 108.3 115.3
 113.7 125.3 133.7 138.7 140.  144.  154.7 155. ]


/Users/heloisefuselier/Dropbox/PDM/Travail/PDM helo/Python/pmc2.py:167: RuntimeWarning: invalid value encountered in arcsin
  self.phyC = arcsin(3*bc/(6*self.Vo+bc))



[68.86724295 52.28621579 -0.9435294  60.13543739 28.73943582 28.73943582]
[ 59.11896216 221.56230667   0.26924417  52.73475766  28.1828752
  28.1828752 ]
0.99955143587754 0.9995549887640355 0.9996813647609688


in this situation $\phi_C^1 > \phi_C^2$ in this situation. planes P1 and P2 in zone 0, 2 and 4 do not intersect

if $\phi_E^1 > \phi_E^2$ planes P1 and P2 in zone 1, 3 and 5 do not intersect

In [4]:
print('P1 P2\n{} {}\n{} {}'.format(P1.phyC*180/pi,P2.phyC*180/pi,P1.phyE*180/pi,P2.phyE*180/pi))

P1 P2
28.739435822307737 28.18287520430161
28.739435822307737 28.18287520430161


In [5]:
special_case_phyC = P1.phyC > P2.phyC
special_case_phyE = P1.phyE > P2.phyE

In [6]:
print('P1 < P2 ?\nphyC : {}\nphyE : {}'.format(special_case_phyC,special_case_phyE))

P1 < P2 ?
phyC : True
phyE : True


In [7]:
print('V0 P1 : {}\nV0 P2 : {}'.format(P1.Vo,P2.Vo))

V0 P1 : 60.13543738548873
V0 P2 : 52.734757662051784


In [8]:
offset = None
if special_case_phyC:
    offset = 0
elif special_case_phyE:
    offset = 2
else:
    print("you shouldn't be here")
print('offset = {}'.format(offset))

offset = 0


## 3D plot | Calculating intersections

### P1 P2 transition points

In [9]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))

for i in range(6):
    eq_pts_transisiton_P12[(i-offset)%6] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])
    
ones = np.ones((6,3))

pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()
print(pts_transisiton_P12)

[[-666.2237267 -666.2237267 -272.6279063 -272.6279063 -272.6279063
  -666.2237267]
 [-272.6279063 -272.6279063 -272.6279063 -666.2237267 -666.2237267
  -666.2237267]
 [-272.6279063 -666.2237267 -666.2237267 -666.2237267 -272.6279063
  -272.6279063]]


In [10]:
if offset == 0:
    pts_transisiton_P12 = pts_transisiton_P12[:,1::2]
elif offset == 2:
    pts_transisiton_P12 = pts_transisiton_P12[:,::2]
print(pts_transisiton_P12)
#assert(pts_transisiton_P12.shape==(3,3))

[[-666.2237267 -272.6279063 -666.2237267]
 [-272.6279063 -666.2237267 -666.2237267]
 [-666.2237267 -666.2237267 -272.6279063]]


$V_0^2$

In [11]:
eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

### Intersection with far plane

In [12]:
def p_plane_intersection_12(P1,P2,plane_dist,offset):
    p_plane_n = np.ones(3)/np.sqrt(3)
    p_plane_o = p_plane_n*plane_dist
    
    eq = np.zeros((12,3,3))
    b = np.array([[1,1,np.dot(p_plane_n,p_plane_o)]])
    
    for i in range(12):
        eq[i] = np.array([get_plane_normal_12_cycle(P1,P2,i,offset),get_plane_normal_12_cycle(P1,P2,i-1,offset),p_plane_n])
        
    pts = np.linalg.solve(eq,b).transpose()
    return pts
    
    

In [13]:
sig_pts = np.concatenate((P1.pts,P2.pts),axis=1)
all_pts = np.concatenate((sig_pts,pts_transisiton_P12,pts_P2),axis=1) #miss pts_P2
pts_with_largest_coord = 1*all_pts[:,np.argmax(np.max(all_pts, axis=0))]
pts_with_largest_coord *= 1.1

In [14]:
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)
pts_far = p_plane_intersection_12(P1,P2,dist,offset)

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.set_aspect('equal')
ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')

In [16]:
ax.plot(*pts_transisiton_P12[:,:], 'ro',alpha=1)
#ax.plot(*pts_transisiton_P12[:,-1:], 'yo',alpha=1)
ax.plot(*pts_P2,'ko',alpha=1)
ax.plot(*pts_far,'bo',alpha=1)

In [17]:
def get_verts(cord):
    return [list(zip(*cord))]

## Draw P2 planes

In [18]:
for i in range(3):
    cord_l = np.zeros((3,4))
    cord_r = np.zeros((3,4))
    far_pts_inter_P2_idx = i*4
    cord_r[:,0] = pts_P2[:,0]
    cord_r[:,1] = pts_transisiton_P12[:,i] 
    cord_r[:,2] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    cord_l[:,0] = pts_P2[:,0]
    cord_l[:,1] = pts_transisiton_P12[:,(i-1)%3] 
    cord_l[:,2] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,3] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 planes

In [19]:
for i in range(3):
    cord_l = np.zeros((3,3))
    cord_r = np.zeros((3,3))
    far_pts_inter_P2_idx = i*4 + 2
    cord_r[:,0] = pts_transisiton_P12[:,i] 
    cord_r[:,1] = pts_far[:,(far_pts_inter_P2_idx+1)%12]
    cord_r[:,2] = pts_far[:,far_pts_inter_P2_idx]

    cord_l[:,0] = pts_transisiton_P12[:,i] 
    cord_l[:,1] = pts_far[:,(far_pts_inter_P2_idx-1)%12]
    cord_l[:,2] = pts_far[:,far_pts_inter_P2_idx]
    
    verts = get_verts(cord_r)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)
    
    verts = get_verts(cord_l)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.2)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [20]:
#ax.set_aspect('equal')
fig.show()

# 2D view

In [22]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [23]:
new_sig_pts = new_base_transform @ sig_pts
new_P2 = new_base_transform @ pts_P2
new_transi = new_base_transform @ pts_transisiton_P12
new_far = new_base_transform @ pts_far

In [24]:
transi_dist = new_transi[2,2]
near_inter = p_plane_intersection_6(P2,transi_dist*0.99)
new_near = new_base_transform @ near_inter

In [25]:
plt.close()
#plt.axis('equal')
plt.plot(new_P2[0,:],new_P2[1,:],'r.')
plt.plot(new_sig_pts[0,:],new_sig_pts[1,:],'k.')
plt.fill(new_far[0,:],new_far[1,:],edgecolor='b',fill=False)
plt.fill(new_near[0,:],new_near[1,:],edgecolor='r',fill=False)

In [ ]:
P1.sol

In [39]:
P2.sol

array([29.88443564, 20.27825491, -1.12435306, 18.67837649, 39.16551874,
       41.50962047])